# Load translation voxel simulation data

In [1]:
%%time
%load_ext autoreload

from analysis_utils import *
from joblib import Parallel, delayed
import multiprocessing  
import numpy as np
import pandas as pd

#Growth rate = 3.0
#Computes the translation latencies from our model with simulations containing
#1 to 14 cognate ternary complexes in the voxel (14 to 42 not included because
#the probability of voxels with that number of cognates is ~0)
path = "/Users/Akshay/Documents/tRNAShuffle/data/"
ensmbl_latency_dict = dict()
rxndiff=dict()
scalingFactor = 200
NR_scaledrates = {'k1r':718*200/scalingFactor,'k2f':1475*200/scalingFactor,'k2r_nr':1120*200/scalingFactor,'k3_nr':6*200/scalingFactor,'k4':209*200/scalingFactor}


data = "191003_2210/"
for i in range(1,7):
    sim_i = computeTransportRxnTimes(path+data,100*(i-1),100*(i-1)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))

data = "191105_0908/"
for i in range(7,14):
    sim_i = computeTransportRxnTimes(path+data, 100*(i-1-6),100*(i-1-6)+100,cogtRNANum=i, ribosomeNum = 7,scaling=scalingFactor,NR_scaling = NR_scaledrates)
    ensmbl_latency_dict[i] = (CellLatencies(sim_i))
    
#Experimentally obtained codon usage frequencies and tRNA abundances at growth rate = 2.5 dbl/hr
gr25_Codon = np.array([2.36, 1.26, 45.55, 34.17, 16.97, 57.86, 19.27, 33.74, 14.98, 22.31, 43.18, 7.67, 24.11, 24.87, 39.49, 11.81, 0.03, 0.63, 2.19, 9.31, 17.22, 55.01, 5.61, 29.21, 21.67, 0.52, 15.79, 43.86, 4.17, 2.61, 20.64, 26.7, 7.03, 0.19, 2.76, 3.81, 6.72, 16.52, 4.27, 2.73, 7.92, 23.25, 2.51, 1.98, 16.33, 11.68, 0.62, 0.67, 43.82, 20.59, 27.28, 7.01, 6.78, 14.21, 60.75, 0.82, 3.86, 4.09, 28.82, 5.18, 4.38, 1.09])
gr25_pCodon = gr25_Codon/np.sum(gr25_Codon)
gr25_ptRNA = [0.0602409638554217, 0.010542168674698798, 0.07379518072289158, 0.00602409638554217, 0.010542168674698798, 0.00602409638554217, 0.021084337349397596, 0.043674698795180725, 0.021084337349397596, 0.01204819277108434, 0.01807228915662651, 0.08433734939759038, 0.03162650602409639, 0.07228915662650603, 0.01204819277108434, 0.07078313253012049, 0.06325301204819278, 0.01656626506024097, 0.009036144578313254, 0.027108433734939763, 0.010542168674698798, 0.03012048192771085, 0.013554216867469882, 0.015060240963855425, 0.0075301204819277125, 0.010542168674698798, 0.0075301204819277125, 0.003012048192771085, 0.021084337349397596, 0.004518072289156627, 0.01656626506024097, 0.01204819277108434, 0.0015060240963855425, 0.009036144578313254, 0.01656626506024097, 0.019578313253012052, 0.015060240963855425, 0.027108434, 0.05873493975903616, 0.019578313]
gr25_ptRNA = gr25_ptRNA/np.sum(gr25_ptRNA)

n_cores = 16

##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Computing...
Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
Unique transcripts without a Gene identifier:  272
Sequences not available for # genes in Ecocyc DB:  153
Inconsistency in codons:  0
3767
Missing genes in transcriptome_dict in compute_transcript_distributions:  133
CPU times: user 47.9 s, sys: 2.94 s, total: 50.9 s
Wall time: 48 s


## Uniform Random tRNA, WT Codon usage, 100 samples

In [2]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA',ptRNA_list)


CPU times: user 13.3 s, sys: 421 ms, total: 13.7 s
Wall time: 24.1 s


## Uniform Random tRNA, WT Codon usage, 10k samples

In [3]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(10000):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_10ksample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_10ksample_ptRNA',ptRNA_list)


CPU times: user 27min 39s, sys: 11.3 s, total: 27min 51s
Wall time: 44min 6s


## WT tRNA, Uniform Random Codon usage, 100 samples

In [ ]:
%%time
elongt_list = list()
pCodon_list = list()
inputs = list()

syn_codon_list = [[0,1,2,3],[4,5],[6,7],[8,9,10,11],[12,13,14,15],[16,17,46,47,48,49],[18,19,42,43,44,45],
              [20,21],[22,23],[24],[25,26,27],[28,29,30,31],[32],[33],[34,35],[36,37],[38,39,54,55,56,57],[40,41],
              [50,51],[52,53],[58,59,60,61]]
        
for i in range(100):
    pCodon_uniform = np.zeros(len(gr25_pCodon))
    for syn_codons in syn_codon_list:
        pCodon_uniform[syn_codons] = np.random.uniform(min(gr25_Codon),max(gr25_Codon),len(syn_codons))
        pCodon_uniform[syn_codons] = sum(pCodon_transcriptome[syn_codons])/sum(pCodon_uniform[syn_codons])*pCodon_uniform[syn_codons]
    pCodon_list.append(pCodon_uniform)
    inputs.append([gr25_ptRNA, pCodon_uniform,ensmbl_latency_dict])

elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]

np.save('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon',pCodon_list)


## WT tRNA, Uniform Random Codon usage, 10k samples

In [ ]:
%%time
elongt_list = list()
pCodon_list = list()
inputs = list()

syn_codon_list = [[0,1,2,3],[4,5],[6,7],[8,9,10,11],[12,13,14,15],[16,17,46,47,48,49],[18,19,42,43,44,45],
              [20,21],[22,23],[24],[25,26,27],[28,29,30,31],[32],[33],[34,35],[36,37],[38,39,54,55,56,57],[40,41],
              [50,51],[52,53],[58,59,60,61]]
        
for i in range(10000):
    pCodon_uniform = np.zeros(len(pCodon_transcriptome))
    for syn_codons in syn_codon_list:
        pCodon_uniform[syn_codons] = np.random.uniform(min(gr25_Codon),max(gr25_Codon),len(syn_codons))
        pCodon_uniform[syn_codons] = sum(pCodon_transcriptome[syn_codons])/sum(pCodon_uniform[syn_codons])*pCodon_uniform[syn_codons]
    pCodon_list.append(pCodon_uniform)
    inputs.append([gr25_ptRNA, pCodon_uniform,ensmbl_latency_dict])

elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]

np.save('./data/WTtRNA_UniformRandomCodonUsage_10ksample_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_10ksample_pCodon',pCodon_list)


## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT)

In [98]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),'tRNA list: ', ptRNA_list[np.argmin(elongt_list)])


        
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  204.7468593260982 tRNA list:  [0.04097175 0.04220117 0.07840056 0.08061873 0.00812508 0.0272181
 0.07286863 0.00698734 0.02825523 0.03007864 0.08252737 0.03908705
 0.04897578 0.06540679 0.00962458 0.06419479 0.0415891  0.05713749
 0.05548289 0.04655925 0.06072482 0.03401584 0.01396239 0.0833985
 0.05674674 0.02705444 0.0649879  0.01736798 0.01844237 0.02587218
 0.07615736 0.01687463 0.02271396 0.04719114 0.03459146 0.0032517
 0.00883137 0.02900779 0.07583317 0.05462228]
Generation:  100  elongt:  180.88479053429342 tRNA list:  [0.05989368 0.02482716 0.03544915 0.00608565 0.00519211 0.00447962
 0.04700594 0.06214881 0.01257315 0.007848   0.03816644 0.06068883
 0.05778737 0.02478957 0.04262527 0.04176344 0.05293268 0.00223158
 0.04477778 0.00559238 0.00317709 0.03541964 0.00744968 0.02952959
 0.01462341 0.00341406 0.0369241  0.00362517 0.05880865 0.00478224
 0.00172154 0.01242

Generation:  1700  elongt:  165.63843072163098 tRNA list:  [0.05660617 0.0030275  0.0454917  0.00687185 0.00764374 0.00398387
 0.0443117  0.04100576 0.0023357  0.00752891 0.03985903 0.06131674
 0.08177427 0.00636209 0.00759754 0.0737208  0.00180382 0.00251988
 0.07597499 0.00790074 0.00288084 0.06766894 0.0074739  0.02674483
 0.05179254 0.00309572 0.06607992 0.00290714 0.07136925 0.00517706
 0.00170146 0.01099391 0.02492491 0.00533519 0.01201543 0.00209452
 0.00276317 0.00171845 0.00468016 0.05094585]
Generation:  1800  elongt:  165.63843072163098 tRNA list:  [0.05660617 0.0030275  0.0454917  0.00687185 0.00764374 0.00398387
 0.0443117  0.04100576 0.0023357  0.00752891 0.03985903 0.06131674
 0.08177427 0.00636209 0.00759754 0.0737208  0.00180382 0.00251988
 0.07597499 0.00790074 0.00288084 0.06766894 0.0074739  0.02674483
 0.05179254 0.00309572 0.06607992 0.00290714 0.07136925 0.00517706
 0.00170146 0.01099391 0.02492491 0.00533519 0.01201543 0.00209452
 0.00276317 0.00171845 0.0046801

## Genetic algorithm: slower tRNA distributions (Codon usage fixed, WT)

In [99]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='slow')
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', max(elongt_list),'tRNA list: ', ptRNA_list[np.argmax(elongt_list)])

np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  226.298128454574 tRNA list:  [0.02156449 0.03724623 0.02868449 0.04893407 0.03991538 0.02185416
 0.03275673 0.00162713 0.03502709 0.00222362 0.02257815 0.02279118
 0.03190735 0.01036395 0.03380584 0.00721043 0.03768863 0.04231782
 0.01240704 0.01533812 0.04752905 0.03113864 0.01728459 0.0395583
 0.0038256  0.02425963 0.0027816  0.04630539 0.01799604 0.02127398
 0.0230036  0.00985721 0.00341733 0.04003672 0.04580028 0.03860111
 0.04067715 0.00238529 0.01898319 0.01904339]
Generation:  100  elongt:  245.57746865552343 tRNA list:  [0.00698945 0.06783574 0.0056026  0.02750386 0.03665203 0.0224422
 0.0027198  0.00808285 0.0246369  0.0530289  0.01969177 0.00243739
 0.00850085 0.0588725  0.02403253 0.00156008 0.07674853 0.03979749
 0.00877295 0.05637553 0.03052934 0.01368538 0.01149256 0.0119375
 0.00510698 0.01858461 0.00384237 0.04640251 0.0119656  0.05740312
 0.01863849 0.006257

Generation:  1700  elongt:  251.78814240687254 tRNA list:  [0.00259199 0.06301384 0.00418609 0.06178806 0.02981495 0.01786811
 0.00508694 0.0025147  0.00715974 0.07519083 0.00360036 0.00214374
 0.00218041 0.04131784 0.00157457 0.00157457 0.07628336 0.07179573
 0.00217387 0.02325427 0.00727642 0.00245911 0.00720225 0.00390497
 0.01085952 0.06142089 0.00157457 0.03543579 0.00222758 0.06519374
 0.05859555 0.05126672 0.00159834 0.00305364 0.06683296 0.03256376
 0.009404   0.0045711  0.07560819 0.00383692]
Generation:  1800  elongt:  251.8189724921221 tRNA list:  [0.00190365 0.07345248 0.00478004 0.0578975  0.04679273 0.08356264
 0.00156668 0.00293127 0.00651086 0.01893095 0.00419678 0.00249886
 0.00272655 0.03808278 0.0017356  0.0017356  0.01366021 0.03111206
 0.0019234  0.04352035 0.05239339 0.00286648 0.00709829 0.00455186
 0.03463998 0.07159564 0.00165583 0.06209074 0.0025966  0.04572837
 0.03998689 0.04097693 0.0017356  0.07807031 0.00208746 0.05409964
 0.01023485 0.00532833 0.03910979

## Genetic algorithm: faster codon usage distributions (tRNA abundances fixed, WT)

In [9]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt.npy')
pCodon_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, pCodon_list, elongt_list = run_ga_CodonSweep(gr25_ptRNA,pCodon_list,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_pCodon',pCodon_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_fastCodonUsage_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  191.20403884990284
Generation:  100  elongt:  183.8317274791367
Generation:  200  elongt:  183.8317274791367
Generation:  300  elongt:  183.79480130218053
Generation:  400  elongt:  183.00058094871963
Generation:  500  elongt:  183.00058094871963
Generation:  600  elongt:  182.9682133905058
Generation:  700  elongt:  181.89405407246474
Generation:  800  elongt:  181.89405407246474
Generation:  900  elongt:  181.89405407246474
Generation:  1000  elongt:  181.89405407246474
Generation:  1100  elongt:  181.89405407246474
Generation:  1200  elongt:  181.89405407246474
Generation:  1300  elongt:  181.88442465158676
Generation:  1400  elongt:  181.59291600231413
Generation:  1500  elongt:  181.59291600231413
Generation:  1600  elongt:  181.46748086212898
Generation:  1700  elongt:  181.38451951397764
Generation:  1800  elongt:  181.38451951397764
Generation:  1900  elongt:  181.38

## Genetic algorithm: slower codon usage distributions (tRNA abundances fixed, WT)

In [10]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_elongt.npy')
pCodon_list = np.load('./data/WTtRNA_UniformRandomCodonUsage_100sample_pCodon.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, pCodon_list, elongt_list = run_ga_CodonSweep(gr25_ptRNA,pCodon_list,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='slow')
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_elongt',elongt_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_pCodon',pCodon_list)
np.save('./data/WTtRNA_UniformRandomCodonUsage_2ksample_slowCodonUsage_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  197.15079674970096
Generation:  100  elongt:  204.67877264653683
Generation:  200  elongt:  207.69370503330927
Generation:  300  elongt:  208.5881553822576
Generation:  400  elongt:  209.1548761281126
Generation:  500  elongt:  209.9507496043587
Generation:  600  elongt:  210.76652294650046
Generation:  700  elongt:  207.24318243566364
Generation:  800  elongt:  208.94475841032855
Generation:  900  elongt:  211.12963582348067
Generation:  1000  elongt:  210.52570746321175
Generation:  1100  elongt:  211.24092953215046
Generation:  1200  elongt:  210.48032627973268
Generation:  1300  elongt:  210.47866205263128
Generation:  1400  elongt:  209.784447529888
Generation:  1500  elongt:  208.99184589196045
Generation:  1600  elongt:  210.6692604960466
Generation:  1700  elongt:  212.53593727342954
Generation:  1800  elongt:  209.31250488275492
Generation:  1900  elongt:  210.61894

# Minimal code

In [4]:
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt, red20=True)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
Unique transcripts without a Gene identifier:  272
Sequences not available for # genes in Ecocyc DB:  153
Inconsistency in codons:  0
3767
Missing genes in transcriptome_dict in compute_transcript_distributions:  133


## Uniform Random tRNA, minimal code, 100 samples


In [7]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_minimalcode_100sample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode_100sample_ptRNA',ptRNA_list)

CPU times: user 12 s, sys: 52.5 ms, total: 12.1 s
Wall time: 18 s


## Genetic algorithm: faster tRNA distribution with full tRNA, minimal code

In [8]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_minimalcode_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_minimalcode_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  206.14060944009728
Generation:  100  elongt:  184.47065249040617
Generation:  200  elongt:  172.76141694247272
Generation:  300  elongt:  171.97699467004063
Generation:  400  elongt:  170.77511429725126
Generation:  500  elongt:  167.5584628728589
Generation:  600  elongt:  166.85355551029303
Generation:  700  elongt:  166.85355551029303
Generation:  800  elongt:  166.60856577493433
Generation:  900  elongt:  166.60856577493433
Generation:  1000  elongt:  166.60856577493433
Generation:  1100  elongt:  166.60856577493433
Generation:  1200  elongt:  164.92418059150185
Generation:  1300  elongt:  164.54101061324639
Generation:  1400  elongt:  164.49475712293437
Generation:  1500  elongt:  164.49475712293437
Generation:  1600  elongt:  164.49475712293437
Generation:  1700  elongt:  164.49475712293437
Generation:  1800  elongt:  164.49475712293437
Generation:  1900  elongt:  164.

## Genetic algorithm: slower tRNA distribution with full tRNA, minimal code

In [9]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_minimalcode_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_minimalcode_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='slow')
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i,'elongt: ', max(elongt_list))
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0 elongt:  229.79502262616816
Generation:  100 elongt:  243.30179107950988
Generation:  200 elongt:  248.55241118882523
Generation:  300 elongt:  249.73461591316013
Generation:  400 elongt:  250.8934690614592
Generation:  500 elongt:  251.18101657269682
Generation:  600 elongt:  251.7319741362925
Generation:  700 elongt:  251.83228975842928
Generation:  800 elongt:  251.83228975842928
Generation:  900 elongt:  251.87044065674507
Generation:  1000 elongt:  251.9466119058292
Generation:  1100 elongt:  252.14476281671844
Generation:  1200 elongt:  252.14476281671844
Generation:  1300 elongt:  252.14476281671844
Generation:  1400 elongt:  252.14476281671844
Generation:  1500 elongt:  252.23480612657724
Generation:  1600 elongt:  252.24424399888665
Generation:  1700 elongt:  252.25445399941663
Generation:  1800 elongt:  252.25445399941663
Generation:  1900 elongt:  252.25445399941663
CPU tim

## minimal uniform random tRNA, minimal code, 100 samples

In [10]:
%%time

##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt, red20=True)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(ptRNA_red20),max(ptRNA_red20),len(ptRNA_red20))
    ptRNA_uniform[tRNA_indices] = 0
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/minimaltRNA_minimalcode_100sample_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode_100sample_ptRNA',ptRNA_list)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
Unique transcripts without a Gene identifier:  272
Sequences not available for # genes in Ecocyc DB:  153
Inconsistency in codons:  0
3767
Missing genes in transcriptome_dict in compute_transcript_distributions:  133


/Users/akshay/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


CPU times: user 2min 26s, sys: 604 ms, total: 2min 26s
Wall time: 2min 32s


## Genetic algorithm: faster tRNA distribution with minimal tRNA, minimal code

In [115]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt, red20=True)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='fast',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        print(ptRNA_list[0])
        
np.save('./data/minimaltRNA_minimalcode_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/minimaltRNA_minimalcode_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0  elongt:  168.09758757389093
[0.094584   0.         0.07732972 0.         0.         0.
 0.09288514 0.00731383 0.05229929 0.         0.01686421 0.12535952
 0.01398833 0.         0.05215043 0.10667536 0.         0.
 0.05866316 0.         0.         0.11496429 0.05205781 0.00121284
 0.06831448 0.         0.         0.         0.08328057 0.
 0.         0.         0.09280249 0.         0.         0.
 0.11712083 0.12272277 0.         0.09683255]
Generation:  100  elongt:  159.35541810875426
[0.1116884  0.         0.06227591 0.         0.         0.
 0.04887724 0.03701001 0.01312146 0.         0.01836093 0.06510311
 0.05769327 0.         0.01661511 0.08410263 0.         0.
 0.11905434 0.         0.         0.0403119  0.05249154 0.01653483
 0.01836093 0.         0.         0.         0.06233005 0.
 0.         0.         0.01551234 0.         0.         0.
 0.01965386 0.04941978 0.         0.09148236]
Generation:  200  elongt:  158.69193124156322
[0.09502171 0.         0.0358948

KeyboardInterrupt: 

## Genetic algorithm: slower tRNA distribution with minimal tRNA, minimal code

In [113]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt, red20=True)
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='slow',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i,'elongt: ', max(elongt_list))
        print(ptRNA_list[0])

np.save('./data/minimaltRNA_minimalcode_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/minimaltRNA_minimalcode_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0 elongt:  200.61332867794016
[0.094584   0.         0.07732972 0.         0.         0.
 0.09288514 0.00731383 0.05229929 0.         0.01686421 0.12535952
 0.01398833 0.         0.05215043 0.10667536 0.         0.
 0.05866316 0.         0.         0.11496429 0.05205781 0.00121284
 0.06831448 0.         0.         0.         0.08328057 0.
 0.         0.         0.09280249 0.         0.         0.
 0.11712083 0.12272277 0.         0.09683255]
Generation:  100 elongt:  211.36661478591012
[0.01221374 0.         0.02760015 0.         0.         0.
 0.02099455 0.03402786 0.11692453 0.         0.07170514 0.01233625
 0.01251068 0.         0.11061682 0.03994041 0.         0.
 0.01357687 0.         0.         0.0313768  0.04008047 0.07912909
 0.07405006 0.         0.         0.         0.08182311 0.
 0.         0.         0.03405611 0.         0.         0.
 0.11190137 0.03066772 0.         0.04446828]


KeyboardInterrupt: 

## minimal uniform random tRNA, minimal code GFP, 100 samples

In [5]:
%%time

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),len(ptRNA_red20))
    ptRNA_uniform[tRNA_indices] = 0
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_GFP,ensmbl_latency_dict])
    
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA',ptRNA_list)
print(np.average(elongt_list))

/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


181.7079011198748
CPU times: user 1.81 s, sys: 72.4 ms, total: 1.88 s
Wall time: 14.6 s


In [54]:
min(ptRNA_red20),max(ptRNA_red20)

(0.0, 0.12824427480759398)

In [2]:
gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')

minRange = min(ptRNA_red20[np.nonzero(ptRNA_red20)])
maxRange = max(ptRNA_red20)

a = ptRNA_list[np.argmin(elongt_list)]
print(a,sum(a))
print(computeElongationLatency(a,pCodon_GFP,ensmbl_latency_dict))
a = np.where((a>0)&(a<minRange), minRange, a)
a = np.where(a>maxRange, maxRange, a)
a_minmaxindices = np.where((a==minRange) | (a==maxRange))
a_indices = np.where((a>minRange) & (a<maxRange))
a[a_indices] = (a[a_indices]/sum(a[a_indices]))*(1-sum(a[a_minmaxindices]))
print(a,sum(a))
print(computeElongationLatency(a,pCodon_GFP,ensmbl_latency_dict))


NameError: name 'ptRNA_red20' is not defined

## Genetic algorithm: faster tRNA distribution with minimal tRNA, GFP

In [8]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *
gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()
### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='fast',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),np.average(elongt_list),'tRNA list: ',ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0  elongt:  171.75080283438695 434.57787740681766 tRNA list:  [0.03367461 0.         0.05628633 0.         0.         0.
 0.03022968 0.05726129 0.02890398 0.         0.02221581 0.03221715
 0.06743309 0.         0.01811383 0.0896722  0.         0.
 0.07110743 0.         0.         0.07108271 0.09471045 0.07212746
 0.01329183 0.         0.         0.         0.05827319 0.
 0.         0.         0.07233203 0.         0.         0.
 0.01744351 0.0536624  0.         0.03996101]
Generation:  100  elongt:  159.89768777287156 163.40984370652868 tRNA list:  [0.02690399 0.         0.05173235 0.         0.         0.
 0.02328147 0.05739122 0.01221374 0.         0.01221374 0.05777824
 0.12068634 0.         0.03182723 0.0608456  0.         0.
 0.10107867 0.         0.         0.1005877  0.02983742 0.04378283
 0.01235338 0.         0.         0.         0.05185915 0.
 0.         0.         0.09850267 0.         0.         0.
 0.01402693 0.03247801 0.         0.06061932]
Generation:  200

Generation:  1700  elongt:  157.19556326201047 159.15672749388773 tRNA list:  [0.0190201  0.         0.04156696 0.         0.         0.
 0.04819245 0.08909173 0.01373045 0.         0.0225166  0.06683728
 0.07671548 0.         0.04067388 0.05614344 0.         0.
 0.11481016 0.         0.         0.09177696 0.01439167 0.04798097
 0.01445198 0.         0.         0.         0.03206805 0.
 0.         0.         0.07992985 0.         0.         0.
 0.01235599 0.03965668 0.         0.07808933]
Generation:  1800  elongt:  157.19556326201047 158.90407288679646 tRNA list:  [0.0190201  0.         0.04156696 0.         0.         0.
 0.04819245 0.08909173 0.01373045 0.         0.0225166  0.06683728
 0.07671548 0.         0.04067388 0.05614344 0.         0.
 0.11481016 0.         0.         0.09177696 0.01439167 0.04798097
 0.01445198 0.         0.         0.         0.03206805 0.
 0.         0.         0.07992985 0.         0.         0.
 0.01235599 0.03965668 0.         0.07808933]
Generation: 

## Genetic algorithm: slower tRNA distribution with minimal tRNA, GFP

In [6]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/sfGFP_RED20.csv')
ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = np.array(ptRNA_red20[6]/sum(ptRNA_red20[6]))
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-sfGFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20[np.nonzero(ptRNA_red20)]),max(ptRNA_red20),objective='slow',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(max(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', max(elongt_list),'tRNA list: ',ptRNA_list[np.argmax(elongt_list)])
        
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-sfGFP_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Generation:  0  elongt:  193.14207697602257 tRNA list:  [0.0337419  0.         0.10416162 0.         0.         0.
 0.08197971 0.01281265 0.07610775 0.         0.11226017 0.08626951
 0.03989334 0.         0.08062111 0.09833604 0.         0.
 0.06056075 0.         0.         0.02810242 0.08690896 0.04011888
 0.08613279 0.         0.         0.         0.03471656 0.
 0.         0.         0.07326589 0.         0.         0.
 0.11683489 0.05830633 0.         0.02794481]
Generation:  100  elongt:  212.77472739971392 tRNA list:  [0.04033359 0.         0.0363369  0.         0.         0.
 0.01449324 0.02086474 0.06319654 0.         0.12207832 0.02154472
 0.02743971 0.         0.02158262 0.01665795 0.         0.
 0.0317577  0.         0.         0.01852204 0.12824427 0.02756128
 0.12062871 0.         0.         0.         0.01774003 0.
 0.         0.         0.01456085 0.         0.         0.
 0.10267459 0.12843    0.         0.02535219]
Generation:  200  elongt:  217.92488749165895 tRNA lis

Generation:  1800  elongt:  222.80357527453845 tRNA list:  [0.12587895 0.         0.02040233 0.         0.         0.
 0.012931   0.01430288 0.12653416 0.         0.10853045 0.01750406
 0.01929032 0.         0.02444435 0.01560992 0.         0.
 0.02103921 0.         0.         0.02237707 0.12679654 0.01324014
 0.12510368 0.         0.         0.         0.01829437 0.
 0.         0.         0.01482501 0.         0.         0.
 0.12653416 0.03103682 0.         0.01532456]
Generation:  1900  elongt:  222.80357527453845 tRNA list:  [0.12587895 0.         0.02040233 0.         0.         0.
 0.012931   0.01430288 0.12653416 0.         0.10853045 0.01750406
 0.01929032 0.         0.02444435 0.01560992 0.         0.
 0.02103921 0.         0.         0.02237707 0.12679654 0.01324014
 0.12510368 0.         0.         0.         0.01829437 0.
 0.         0.         0.01482501 0.         0.         0.
 0.12653416 0.03103682 0.         0.01532456]
CPU times: user 3min 37s, sys: 3.09 s, total: 3min

In [48]:
print(min(ptRNA_red20[np.nonzero(ptRNA_red20)]))
print(max(ptRNA_red20))

0.012213740457866095
0.12824427480759398


## minimal uniform random tRNA, minimal code mRFP, 100 samples

In [43]:
%%time

gene_map, pCodon_GFP = singlegene_to_genemap('/Users/akshay/Documents/tRNAShuffle/data/tables/mRFP_RED20.csv')

ptRNA_red20 = pd.read_excel('./data/tables/tRNAValues_RED20.xlsx',header=None)
ptRNA_red20 = ptRNA_red20[6]/sum(ptRNA_red20[6])
tRNA_indices = np.argwhere(ptRNA_red20 == 0)
tRNA_indices = [indices[0] for indices in tRNA_indices]
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(ptRNA_red20),max(ptRNA_red20),len(ptRNA_red20))
    ptRNA_uniform[tRNA_indices] = 0
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_GFP,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt',elongt_list)
np.save('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA',ptRNA_list)
print(np.average(elongt_list))

184.3356414849228
CPU times: user 12.4 s, sys: 354 ms, total: 12.7 s
Wall time: 22 s


## Genetic algorithm: faster tRNA distribution with minimal tRNA, mRFP

In [44]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20),max(ptRNA_red20),objective='fast',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  168.74550105211244
Generation:  100  elongt:  160.17600220335808
Generation:  200  elongt:  156.65755391893552
Generation:  300  elongt:  155.83525207164303
Generation:  400  elongt:  155.79226660354004
Generation:  500  elongt:  155.31931077776082
Generation:  600  elongt:  154.47035931724673
Generation:  700  elongt:  154.47035931724673
Generation:  800  elongt:  154.34752486159513
Generation:  900  elongt:  154.34752486159513
Generation:  1000  elongt:  154.34752486159513
Generation:  1100  elongt:  153.24877589258833
Generation:  1200  elongt:  153.24877589258833
Generation:  1300  elongt:  153.24877589258833
Generation:  1400  elongt:  153.24877589258833
Generation:  1500  elongt:  153.24877589258833
Generation:  1600  elongt:  153.24877589258833
Generation:  1700  elongt:  153.24877589258833
Generation:  1800  elongt:  153.24877589258833
Generation:  1900  elongt:  153

## Genetic algorithm: slower tRNA distribution with minimal tRNA, mRFP

In [45]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_elongt.npy')
ptRNA_list = np.load('./data/minimaltRNA_minimalcode-mRFP_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_GFP,elongt_list,ensmbl_latency_dict,min(ptRNA_red20),max(ptRNA_red20),objective='slow',ptRNA_red20=ptRNA_red20)
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list))
        
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_minimalcode-mRFP_2ksample_slowtRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  175.1247041405708
Generation:  100  elongt:  211.03438110718938
Generation:  200  elongt:  222.29216190704705
Generation:  300  elongt:  229.20293389041973
Generation:  400  elongt:  222.00841180441049
Generation:  500  elongt:  222.42975185073902
Generation:  600  elongt:  235.79605392618652
Generation:  700  elongt:  232.7473907291243
Generation:  800  elongt:  235.0525272570431
Generation:  900  elongt:  236.13680000893967
Generation:  1000  elongt:  238.68686851934845
Generation:  1100  elongt:  233.47878212946296
Generation:  1200  elongt:  233.2166650611879
Generation:  1300  elongt:  238.0814716424852
Generation:  1400  elongt:  235.79969613071896
Generation:  1500  elongt:  205.83571058026317
Generation:  1600  elongt:  235.51616418980637
Generation:  1700  elongt:  240.72484231501417
Generation:  1800  elongt:  240.78492448637914
Generation:  1900  elongt:  238.6056

In [9]:
import numpy as np
gr25_Codon = np.array([2.36, 1.26, 45.55, 34.17, 16.97, 57.86, 19.27, 33.74, 14.98, 22.31, 43.18, 7.67, 24.11, 24.87, 39.49, 11.81, 0.03, 0.63, 2.19, 9.31, 17.22, 55.01, 5.61, 29.21, 21.67, 0.52, 15.79, 43.86, 4.17, 2.61, 20.64, 26.7, 7.03, 0.19, 2.76, 3.81, 6.72, 16.52, 4.27, 2.73, 7.92, 23.25, 2.51, 1.98, 16.33, 11.68, 0.62, 0.67, 43.82, 20.59, 27.28, 7.01, 6.78, 14.21, 60.75, 0.82, 3.86, 4.09, 28.82, 5.18, 4.38, 1.09])
gr25_pCodon = gr25_Codon/np.sum(gr25_Codon)
gr25_ptRNA = [0.0602409638554217, 0.010542168674698798, 0.07379518072289158, 0.00602409638554217, 0.010542168674698798, 0.00602409638554217, 0.021084337349397596, 0.043674698795180725, 0.021084337349397596, 0.01204819277108434, 0.01807228915662651, 0.08433734939759038, 0.03162650602409639, 0.07228915662650603, 0.01204819277108434, 0.07078313253012049, 0.06325301204819278, 0.01656626506024097, 0.009036144578313254, 0.027108433734939763, 0.010542168674698798, 0.03012048192771085, 0.013554216867469882, 0.015060240963855425, 0.0075301204819277125, 0.010542168674698798, 0.0075301204819277125, 0.003012048192771085, 0.021084337349397596, 0.004518072289156627, 0.01656626506024097, 0.01204819277108434, 0.0015060240963855425, 0.009036144578313254, 0.01656626506024097, 0.019578313253012052, 0.015060240963855425, 0.027108434, 0.05873493975903616, 0.019578313]
gr25_ptRNA = gr25_ptRNA/np.sum(gr25_ptRNA)

max(gr25_ptRNA)

0.08523592085132133

# Stratified high analysis

In [12]:
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt,stratify='high')
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
high
Unique transcripts without a Gene identifier:  0
Sequences not available for # genes in Ecocyc DB:  1
Inconsistency in codons:  0
419
Missing genes in transcriptome_dict in compute_transcript_distributions:  3135


## Uniform Random tRNA, WT Codon usage, Stratify high, 100 samples

In [19]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_100sample_elongt2',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_100sample_ptRNA2',ptRNA_list)


CPU times: user 1.02 s, sys: 119 ms, total: 1.14 s
Wall time: 5.27 s


## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT, stratify high)

In [20]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_100sample_elongt2.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_100sample_ptRNA2.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),'tRNA list: ', ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_2ksample_fasttRNA_elongt2',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_2ksample_fasttRNA_ptRNA2',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhigh_2ksample_fasttRNA_fullelongt2',fullelongt_list)
#181.67441493531658
#182.98327531311577

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  199.9037713899387 tRNA list:  [0.07048057 0.00810124 0.0708845  0.04052682 0.01879943 0.04622592
 0.03639752 0.07123616 0.03365665 0.08446451 0.02645896 0.06893499
 0.0043163  0.08501174 0.04892115 0.07454627 0.05838645 0.03267077
 0.0269581  0.03428346 0.0314576  0.07401035 0.04732401 0.05245348
 0.03653849 0.0025605  0.01924943 0.03019563 0.00161054 0.0057083
 0.00441681 0.07017546 0.07708379 0.01479219 0.0197264  0.00488213
 0.0139003  0.02128221 0.00572162 0.07019199]
Generation:  100  elongt:  186.75780992125803 tRNA list:  [0.06593568 0.04720341 0.04436855 0.00683762 0.00663098 0.01933121
 0.03250036 0.04993901 0.02178414 0.00640306 0.01017111 0.0514152
 0.00615683 0.05322931 0.00318596 0.05991788 0.00575576 0.00846727
 0.06320007 0.0053006  0.00937814 0.02570879 0.01320197 0.04910308
 0.0113881  0.00320918 0.00765257 0.02510654 0.00210618 0.01082277
 0.00559036 0.0058

Generation:  1700  elongt:  175.61430694843128 tRNA list:  [0.08206052 0.00675781 0.06433686 0.00568795 0.00443208 0.00994328
 0.05952468 0.07765896 0.00908987 0.00336249 0.01580149 0.07451418
 0.00588941 0.06969712 0.00321541 0.0693407  0.02004763 0.00622291
 0.06612271 0.00345258 0.00788443 0.060222   0.01008628 0.00639431
 0.01071142 0.00223827 0.02930515 0.00674703 0.00152207 0.00871061
 0.01588858 0.00517914 0.03774215 0.00450161 0.02212687 0.01549405
 0.00698459 0.01006655 0.05547706 0.02555921]
Generation:  1800  elongt:  175.61430694843128 tRNA list:  [0.08206052 0.00675781 0.06433686 0.00568795 0.00443208 0.00994328
 0.05952468 0.07765896 0.00908987 0.00336249 0.01580149 0.07451418
 0.00588941 0.06969712 0.00321541 0.0693407  0.02004763 0.00622291
 0.06612271 0.00345258 0.00788443 0.060222   0.01008628 0.00639431
 0.01071142 0.00223827 0.02930515 0.00674703 0.00152207 0.00871061
 0.01588858 0.00517914 0.03774215 0.00450161 0.02212687 0.01549405
 0.00698459 0.01006655 0.0554770

# Stratified low analysis

In [2]:
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt,stratify='low')
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
low
Unique transcripts without a Gene identifier:  87
Sequences not available for # genes in Ecocyc DB:  15
Inconsistency in codons:  0
319
Missing genes in transcriptome_dict in compute_transcript_distributions:  3552


## Uniform Random tRNA, WT Codon usage, Stratify low, 100 samples

In [6]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_100sample_elongt2',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_100sample_ptRNA2',ptRNA_list)


CPU times: user 972 ms, sys: 107 ms, total: 1.08 s
Wall time: 5.18 s


## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT, stratify low)

In [7]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_100sample_elongt2.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_100sample_ptRNA2.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),'tRNA list: ', ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_2ksample_fasttRNA_elongt2',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_2ksample_fasttRNA_ptRNA2',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifylow_2ksample_fasttRNA_fullelongt2',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  205.02217986591748 tRNA list:  [0.05739365 0.02452347 0.05411782 0.0410329  0.06325301 0.06396481
 0.05145258 0.08381041 0.05253797 0.02733521 0.04495403 0.02110141
 0.07295808 0.08267254 0.01551077 0.03220597 0.01933165 0.03860729
 0.07530392 0.04397849 0.02767561 0.08132883 0.02805681 0.04836318
 0.03523914 0.04839127 0.03847096 0.011078   0.0150149  0.04141308
 0.0735473  0.03765651 0.04584569 0.0170185  0.05367809 0.05410025
 0.05104488 0.07819832 0.06618586 0.04712858]
Generation:  100  elongt:  192.72839451553105 tRNA list:  [0.06229472 0.01033428 0.03865035 0.05416503 0.00446923 0.00495768
 0.05587404 0.06382311 0.00398067 0.00200928 0.00916225 0.07600789
 0.01780029 0.03183878 0.00550863 0.00449426 0.00324904 0.01385048
 0.0603513  0.00735626 0.00152962 0.02921757 0.00362565 0.01818448
 0.01821738 0.02036695 0.03658897 0.00770462 0.00555446 0.01361861
 0.04563591 0.0

Generation:  1700  elongt:  180.20115759154515 tRNA list:  [0.04116604 0.00487301 0.05955201 0.00912609 0.00454037 0.00601343
 0.07390584 0.0626484  0.00258425 0.00352413 0.00266754 0.0561836
 0.00548756 0.07628348 0.00376935 0.06243696 0.00588207 0.01223744
 0.05106787 0.00855941 0.00152207 0.06945627 0.00203956 0.00868825
 0.00927706 0.00152207 0.04353404 0.00314364 0.00362662 0.00213017
 0.05168726 0.00795924 0.04821414 0.00511191 0.02976953 0.00378085
 0.01629817 0.04373259 0.03297697 0.06302072]
Generation:  1800  elongt:  180.20115759154515 tRNA list:  [0.04116604 0.00487301 0.05955201 0.00912609 0.00454037 0.00601343
 0.07390584 0.0626484  0.00258425 0.00352413 0.00266754 0.0561836
 0.00548756 0.07628348 0.00376935 0.06243696 0.00588207 0.01223744
 0.05106787 0.00855941 0.00152207 0.06945627 0.00203956 0.00868825
 0.00927706 0.00152207 0.04353404 0.00314364 0.00362662 0.00213017
 0.05168726 0.00795924 0.04821414 0.00511191 0.02976953 0.00378085
 0.01629817 0.04373259 0.03297697 

# Stratified middle analysis

In [6]:
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt,stratify='middle')
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
middle
Unique transcripts without a Gene identifier:  185
Sequences not available for # genes in Ecocyc DB:  138
Inconsistency in codons:  0
3035
Missing genes in transcriptome_dict in compute_transcript_distributions:  550


## Uniform Random tRNA, WT Codon usage, Stratify middle, 100 samples

In [7]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_100sample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_100sample_ptRNA',ptRNA_list)


CPU times: user 992 ms, sys: 120 ms, total: 1.11 s
Wall time: 4.92 s


## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT, stratify middle)

In [8]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),'tRNA list: ', ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifymiddle_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  205.3754195271518 tRNA list:  [0.02252387 0.04710881 0.04466926 0.00487615 0.01892844 0.00373341
 0.03994096 0.04051519 0.03860483 0.00378887 0.00259674 0.03993372
 0.0332039  0.03485956 0.04172088 0.03948993 0.0457827  0.0090039
 0.01835594 0.01508113 0.02964443 0.0322906  0.00503252 0.02785924
 0.02036634 0.01447032 0.02791339 0.01494941 0.00968979 0.03523283
 0.01435726 0.00771904 0.02694867 0.03606587 0.03100204 0.03865371
 0.01183661 0.00729494 0.03036608 0.03358871]
Generation:  100  elongt:  194.01558690722592 tRNA list:  [0.07455976 0.03542896 0.07320964 0.00995677 0.01023491 0.00882236
 0.06242049 0.05031413 0.01079745 0.0020487  0.01351142 0.04283526
 0.00301156 0.0447745  0.02251528 0.05661246 0.0186842  0.03494189
 0.02783951 0.01361468 0.03064923 0.04194094 0.0199993  0.05756731
 0.01285794 0.00941409 0.01758992 0.00808338 0.00838646 0.00228525
 0.00976225 0.014

Generation:  1700  elongt:  183.47354917246565 tRNA list:  [0.06583396 0.01477709 0.07702935 0.00255385 0.0019416  0.00389949
 0.04190283 0.06471302 0.00277563 0.00152207 0.02351    0.06681497
 0.00790421 0.07277289 0.02032019 0.07325135 0.02063763 0.00868838
 0.05243144 0.00310594 0.01652787 0.04459713 0.00215102 0.04641545
 0.00179475 0.0166834  0.0457737  0.00359881 0.00902913 0.00462889
 0.01280179 0.00336927 0.01540943 0.00484188 0.01919237 0.00665585
 0.00309378 0.00512427 0.06661506 0.04531027]
Generation:  1800  elongt:  183.1331048154038 tRNA list:  [0.07239488 0.0041974  0.07119533 0.00292975 0.0080727  0.0041129
 0.0733338  0.07465469 0.00315385 0.00166446 0.02341311 0.07788409
 0.00164793 0.08284207 0.02344192 0.06591361 0.00858288 0.0055972
 0.07986374 0.00821771 0.00171869 0.02106108 0.00221083 0.01539289
 0.01473594 0.0102677  0.05167095 0.00415168 0.00415025 0.00436541
 0.01454958 0.00382926 0.02329484 0.01104749 0.00744879 0.00780666
 0.00356907 0.0053798  0.0572263  0

# Stratified highest analysis

In [2]:
##Compute the codon distribution and elongation latency of each gene weighted by transcript number
codon_elongt = compute_codon_elongt(gr25_ptRNA, len(gr25_pCodon), ensmbl_latency_dict)
gene_map, gene_latency = compute_gene_elongt(codon_elongt,stratify='highest')
pCodon_transcriptome, transcriptome_elongt = compute_transcript_distributions(gene_map,gene_latency)
pCodon_transcriptome = np.array(pCodon_transcriptome)

Number of polypeptides reported:  4434
Returned gene map of length  15934
Removed  24  non-divisible by three genes
['insAB1', 'insCD2', 'dgd', 'insCD4', 'insCD6', 'alaB', 'prfB', 'supK', 'b2891', 'ECK2886', 'insCD3', 'insEF4', 'insAB2', 'insEF5', 'insCD1', 'insAB5', 'insEF1', 'insCD5', 'insEF3', 'insAB3', 'insJK', 'insAB4', 'insEF2', 'insAB6']
Unique transcripts without a Gene identifier:  0
Sequences not available for # genes in Ecocyc DB:  0
Inconsistency in codons:  0
1
Missing genes in transcriptome_dict in compute_transcript_distributions:  3553


## Uniform Random tRNA, WT Codon usage, Stratify highest, 100 samples

In [3]:
%%time
elongt_list = list()
ptRNA_list = list()
inputs = list()

for i in range(100):
    ptRNA_uniform = np.random.uniform(min(gr25_ptRNA),max(gr25_ptRNA),len(gr25_ptRNA))
    ptRNA_list.append(ptRNA_uniform)
    inputs.append([ptRNA_uniform,pCodon_transcriptome,ensmbl_latency_dict])
elongt_list = Parallel(n_jobs=n_cores,backend='loky')(delayed(computeElongationLatency_multithread)(i) for i in inputs)
elongt_list = [elongt[0][0] for elongt in elongt_list]
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_100sample_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_100sample_ptRNA',ptRNA_list)


CPU times: user 1.02 s, sys: 116 ms, total: 1.13 s
Wall time: 4.96 s


## Genetic algorithm: faster tRNA distributions (Codon usage fixed, WT, stratify fastest)

In [4]:
%%time
%load_ext autoreload
%autoreload 2
from analysis_utils import *

### Codon array fixed
elongt_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_100sample_elongt.npy')
ptRNA_list = np.load('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_100sample_ptRNA.npy')
fullelongt_list = list()

for i in range(2000):
    fitness, ptRNA_list, elongt_list = run_ga_tRNA(ptRNA_list,pCodon_transcriptome,elongt_list,ensmbl_latency_dict,min(gr25_ptRNA),max(gr25_ptRNA),objective='fast')
    fullelongt_list.append(min(elongt_list))
    if i%100 == 0:
        print('Generation: ',i, ' elongt: ', min(elongt_list),'tRNA list: ', ptRNA_list[np.argmin(elongt_list)])
        
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_2ksample_fasttRNA_elongt',elongt_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_2ksample_fasttRNA_ptRNA',ptRNA_list)
np.save('./data/UniformRandomtRNA_WTCodonUsage_stratifyhighest_2ksample_fasttRNA_fullelongt',fullelongt_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generation:  0  elongt:  200.8479691584339 tRNA list:  [0.06533194 0.01021503 0.04558136 0.04064659 0.01063363 0.02416097
 0.0541582  0.08449833 0.06131693 0.00766698 0.03732016 0.07882316
 0.00881026 0.08344595 0.07381428 0.02091468 0.03642972 0.06425687
 0.05564017 0.04036498 0.07460494 0.02017291 0.06869501 0.07061022
 0.01502535 0.02198949 0.07569388 0.01776585 0.00162098 0.01170162
 0.04320962 0.05502687 0.04550927 0.04011707 0.0531998  0.08120076
 0.02187229 0.0127736  0.04858387 0.0731898 ]
Generation:  100  elongt:  176.61027935870848 tRNA list:  [0.07253325 0.00389356 0.03342516 0.00544211 0.03430507 0.02325614
 0.05690058 0.05271788 0.00910503 0.02959502 0.01938204 0.01466569
 0.00531966 0.06703732 0.01104767 0.06690533 0.00184519 0.00326017
 0.03514793 0.00771885 0.01982972 0.03253043 0.02603628 0.06135471
 0.00288682 0.00707298 0.01167688 0.00492644 0.01037277 0.00157513
 0.00243007 0.02

Generation:  1700  elongt:  161.8932194267041 tRNA list:  [0.08088572 0.00532297 0.00917371 0.00328306 0.00355354 0.00890304
 0.08523592 0.08523592 0.0064131  0.00834132 0.00858327 0.08217277
 0.0043546  0.07679413 0.00319594 0.00607269 0.01722357 0.0041019
 0.05385432 0.01020014 0.0069119  0.04061555 0.00335868 0.07363719
 0.00729577 0.00180699 0.00483428 0.01804502 0.00792594 0.0015431
 0.00255616 0.0128794  0.03579539 0.00601138 0.03513247 0.01509234
 0.00973043 0.08180197 0.02406223 0.0480622 ]
Generation:  1800  elongt:  161.8932194267041 tRNA list:  [0.08088572 0.00532297 0.00917371 0.00328306 0.00355354 0.00890304
 0.08523592 0.08523592 0.0064131  0.00834132 0.00858327 0.08217277
 0.0043546  0.07679413 0.00319594 0.00607269 0.01722357 0.0041019
 0.05385432 0.01020014 0.0069119  0.04061555 0.00335868 0.07363719
 0.00729577 0.00180699 0.00483428 0.01804502 0.00792594 0.0015431
 0.00255616 0.0128794  0.03579539 0.00601138 0.03513247 0.01509234
 0.00973043 0.08180197 0.02406223 0.04